In [1]:
#installing extra libraries on prod
!pip install google-api-core==2.7.1 google-api-python-client==2.42.0 google-auth==2.6.6
!pip install --upgrade google-api-python-client oauth2client
!pip install Levenshtein==0.17.0
!pip install zeno-etl-libs==1.0.40
!pip install openpyxl==3.0.9

  Using cached google_api_python_client-2.42.0-py2.py3-none-any.whl (8.3 MB)
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 2.51.0
    Uninstalling google-api-python-client-2.51.0:
      Successfully uninstalled google-api-python-client-2.51.0


  Using cached google_api_python_client-2.51.0-py2.py3-none-any.whl (8.6 MB)
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 2.42.0
    Uninstalling google-api-python-client-2.42.0:
      Successfully uninstalled google-api-python-client-2.42.0


In [2]:
"""
Automailer for Playstore Reviews
Fuzzy use to get patient names
author : neha.karekar@zeno.health
"""

'\nAutomailer for Playstore Reviews\nFuzzy use to get patient names\nauthor : neha.karekar@zeno.health\n'

In [3]:
import argparse
import sys
import os
import pandas as pd
import dateutil
import datetime
from dateutil.tz import gettz
import numpy as np
import Levenshtein as lev
from datetime import timedelta

In [4]:
sys.path.append('../../../..')

In [5]:
from zeno_etl_libs.logger import get_logger
from zeno_etl_libs.db.db import DB
from zeno_etl_libs.helper.aws.s3 import S3
from zeno_etl_libs.helper import helper
from zeno_etl_libs.helper.email.email import Email

## Pass Params


In [6]:
env = "dev"
full_run = 0
email_to ="neha.karekar@zeno.health,kuldeep.singh@zeno.health"

In [7]:
os.environ['env'] = env
logger = get_logger()
logger.info(f"full_run: {full_run}")


2022-06-27 10:51:51,266 - root - INFO - full_run: 0


In [8]:
rs_db = DB()
rs_db_write = DB(read_only=False)
rs_db.open_connection()
rs_db_write.open_connection()
s3 = S3()

In [9]:
schema = 'prod2-generico'
table_name = 'ecomm-playstore-patients'
table_info = helper.get_table_info(db=rs_db, table_name=table_name, schema=schema)

In [10]:
# max of data
playstore_q = """
select
            max("review-created-at") max_exp
        from
            "prod2-generico"."ecomm-playstore-reviews" 
        """
max_exp_date = rs_db.get_df(playstore_q)
max_exp_date['max_exp'].fillna(np.nan, inplace=True)
print(max_exp_date.info())
max_exp_date = max_exp_date['max_exp'].to_string(index=False)
print(max_exp_date)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   max_exp  1 non-null      datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 136.0 bytes
None
2022-06-26 12:25:52


In [11]:
# params
if full_run or max_exp_date == 'NaN':
    start = '2022-01-01'
else:
    start = max_exp_date
start = dateutil.parser.parse(start)
print(start)
startminus7 = start - timedelta(days=7)
startminus14 = start - timedelta(days=14)

2022-06-26 12:25:52


In [12]:
# max of data1
playstore_q = """
select
            max("review-created-at") max_exp
        from
            "prod2-generico"."ecomm-playstore-patients" 
        """
max_exp_date1 = rs_db.get_df(playstore_q)
max_exp_date1['max_exp'].fillna(np.nan, inplace=True)
print(max_exp_date1.info())
max_exp_date1 = max_exp_date1['max_exp'].to_string(index=False)
print(max_exp_date1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   max_exp  0 non-null      float64
dtypes: float64(1)
memory usage: 136.0 bytes
None
NaN


In [13]:
# params1
if full_run or max_exp_date1 == 'NaN':
    start1 = '2022-01-01'
else:
    start1 = max_exp_date1
start1 = dateutil.parser.parse(start1)
print(start1)

2022-01-01 00:00:00


In [14]:
#fetching all reviews from playstore-review table
q = f"""
    select
        "review-id",
        "review",
        "author-name",
        "review-created-at",
        "star-rating"
    from
        "prod2-generico"."ecomm-playstore-reviews"
    where
        date("review-created-at")> '{startminus7}'
       """
reviews = rs_db.get_df(q)
reviews.columns = [c.replace('-', '_') for c in reviews.columns]
reviews['review_created_at'] = pd.to_datetime(reviews['review_created_at'])
reviews['review_day_pre7'] = reviews['review_created_at'] - pd.DateOffset(days=7)

In [15]:
#fetching zeno orders near reviews
zeno_q = f"""
    select
        zo.id as zeno_order_id_before_review ,
        zo."patient-id" ,
        zo."created-at" as order_created_at,
        p.phone,
        p."name" as "matched-name"
    from
        "prod2-generico"."zeno-order" zo
    left join "prod2-generico".patients p on
        zo."patient-id" = p.id
    where
        date(zo."created-at") > '{startminus14}'
        and p."name" is not null
    """
zeno_orders = rs_db.get_df(zeno_q)

In [16]:
#Cross join reviews and orders
reviews['i'] = 1
zeno_orders['i'] = 1
merged_df = pd.merge(reviews, zeno_orders, how='outer', on='i')
merged_df['author_name'] = merged_df['author_name'].str.lower()
merged_df['matched-name'] = merged_df['matched-name'].str.lower()


In [17]:
#apply lev fuzzy
merged_df['lev_ratio'] = merged_df.apply(lambda row: lev.ratio(str(row['author_name']),str(row['matched-name'])), 1)
merged_df['rank_order'] = merged_df.sort_values(['zeno_order_id_before_review'], ascending=[False]) \
                              .groupby(['review_id', 'matched-name']) \
                              .cumcount() + 1
latest_order = merged_df[(merged_df['rank_order'] == 1)]
latest_order.columns
latest_order['top_3_matches'] = latest_order.sort_values(['lev_ratio'], ascending=[False]).groupby(['review_id']) \
                                    .cumcount() + 1

c:\users\lenovo\documents\generico-projects\etl\etl_env\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == "__main__":


In [18]:
#fetch top 3 name matches
latest_order = latest_order[(latest_order['top_3_matches'] <= 3)]
latest_order = latest_order.sort_values(['star_rating', 'review_id', 'top_3_matches']
                                        , ascending=[True, True, True])

In [19]:
#making two dataframes 1 for storing 1 for mailing
latest_order.columns = [c.replace('_', '-') for c in latest_order.columns]
latest_order_data = latest_order[['review-id', 'review', 'star-rating', 'review-created-at', 'author-name',
                                  'matched-name', 'zeno-order-id-before-review', 'patient-id'
    , 'order-created-at', 'phone', 'lev-ratio']]
latest_order_mail = latest_order[['review-id', 'review', 'star-rating', 'review-created-at', 'author-name',
                                  'matched-name', 'zeno-order-id-before-review', 'patient-id'
    , 'order-created-at']]

In [20]:
latest_order_data['created-at'] = datetime.datetime.now(tz=gettz('Asia/Kolkata')).strftime('%Y-%m-%d %H:%M:%S')
latest_order_data['updated-at'] = datetime.datetime.now(tz=gettz('Asia/Kolkata')).strftime('%Y-%m-%d %H:%M:%S')
latest_order_data['created-by'] = 'etl-automation'
latest_order_data['updated-by'] = 'etl-automation'
latest_order_data=latest_order_data[(pd.to_datetime(latest_order_data['review-created-at']) > start1)]
latest_order_mail=latest_order_mail[(pd.to_datetime(latest_order_mail['review-created-at']) > start1)]

c:\users\lenovo\documents\generico-projects\etl\etl_env\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
c:\users\lenovo\documents\generico-projects\etl\etl_env\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\users\lenovo\documents\generico-projects\etl\etl_env\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

In [21]:
if latest_order_mail.empty:
    print('DataFrame is empty!')
    exit()

In [22]:
if isinstance(table_info, type(None)):
    raise Exception(f"table: {table_name} do not exist, create the table first")
else:
    logger.info(f"Table:{table_name} exists")

2022-06-27 10:52:48,749 - root - INFO - Table:ecomm-playstore-patients exists


In [23]:
truncate_query = f''' DELETE FROM "{schema}"."{table_name}" where "review-created-at" >'{start1}' '''
print(truncate_query)
rs_db_write.execute(truncate_query)

 DELETE FROM "prod2-generico"."ecomm-playstore-patients" where "review-created-at" >'2022-01-01 00:00:00' 


In [24]:
s3.write_df_to_db(df=latest_order_data[table_info['column_name']], table_name=table_name, db=rs_db_write,
                  schema=schema)
file_name = 'Zeno_playstore.xlsx'
file_path = s3.write_df_to_excel(data={'Zeno Playstore': latest_order_mail}, file_name=file_name)

email = Email()
# file_path ='/Users/Lenovo/Downloads/utter.csv'
email.send_email_file(subject="Zeno Playstore",
                      mail_body='Zeno Playstore',
                      to_emails=email_to, file_uris=[], file_paths=[file_path])

S3 object(uri: s3://aws-glue-temporary-921939243643-ap-south-1/temp_1656307378032.csv) delete response: {'ResponseMetadata': {'RequestId': '0831P67A934W13EB', 'HostId': 'dvWeTvkiRUEufGtM//X/wb47gg4xfAH2WLfcJvRqOJM6du6p31uCqtbnWxvzG9Mlyz0m9H69uyo=', 'HTTPStatusCode': 204, 'HTTPHeaders': {'x-amz-id-2': 'dvWeTvkiRUEufGtM//X/wb47gg4xfAH2WLfcJvRqOJM6du6p31uCqtbnWxvzG9Mlyz0m9H69uyo=', 'x-amz-request-id': '0831P67A934W13EB', 'date': 'Mon, 27 Jun 2022 05:23:13 GMT', 'server': 'AmazonS3'}, 'RetryAttempts': 0}}
S3 object(uri: s3://aws-glue-temporary-921939243643-ap-south-1/temp_1656307378032.csv) delete response: {'ResponseMetadata': {'RequestId': 'CACKE2ANXR5MRXNV', 'HostId': 'qz0HodyA0kE/Tama58zckZ+WqxCl+JL6/mDTzE3Tp2SyfC8hjgBGd4t5nHd9yKDJLNmvbYaCl20=', 'HTTPStatusCode': 204, 'HTTPHeaders': {'x-amz-id-2': 'qz0HodyA0kE/Tama58zckZ+WqxCl+JL6/mDTzE3Tp2SyfC8hjgBGd4t5nHd9yKDJLNmvbYaCl20=', 'x-amz-request-id': 'CACKE2ANXR5MRXNV', 'date': 'Mon, 27 Jun 2022 05:23:14 GMT', 'server': 'AmazonS3'}, 'RetryA

In [25]:
# Closing the DB Connection
rs_db.close_connection()

Redshift DB connection closed successfully.
